In [ ]:
# Re-import necessary libraries due to execution reset
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Define final categorization for Steering_Controls
def categorize_steering_controls_final(value):
    if value in ["Four Wheel Standard", "Conventional"]:
        return "Mid Price Steering"
    elif value in ["Command Control"]:
        return "High Price Steering"
    else:
        return "Unknown"



# Define price-based categories for Track_Type
def categorize_track_type(value):
    if value in ["Rubber"]:
        return "Low Price Track"
    elif value in ["Steel"]:
        return "High Price Track"
    else:
        return "Unknown"


# Define price-based categories for Transmission
def categorize_transmission(value):
    if value in ["Powershuttle", "Standard", "Direct Drive"]:
        return "Low Price Transmission"
    elif value in ["Autoshift", "Hydrostatic"]:
        return "Mid Price Transmission"
    elif value in ["Powershift", "None or Unspecified", "AutoShift"]:
        return "High Price Transmission"
    else:
        return "Unknown"


# Define price-based categories for Steering_Controls
def categorize_steering_controls(value):
    if value in ["Four Wheel Standard"]:
        return "Low Price Steering"
    elif value in ["Conventional"]:
        return "Mid Price Steering"
    elif value in ["Command Control"]:
        return "High Price Steering"
    else:
        return "Unknown"



# Define price-based categories for ProductGroup
def categorize_product_group(price):
    if price < 20000:
        return "Low Price Group"
    elif 20000 <= price < 40000:
        return "Mid Price Group"
    else:
        return "High Price Group"


def create_model_category_mapping(training_df):
    """
    Creates a mapping of ModelID to Model_Category based on fiModelDesc and SalePrice in training data.
    
    Parameters:
    training_df (pd.DataFrame): Training dataset containing fiModelDesc and SalePrice.
    
    Returns:
    dict: A dictionary mapping ModelID to Model_Category.
    """
    # Compute average SalePrice per fiModelDesc in training data
    model_avg_price = training_df.groupby("fiModelDesc")["SalePrice"].mean()
    
    # Define price categories
    def categorize_price(price):
        if price < 20000:
            return "Low Price Models"
        elif 20000 <= price < 60000:
            return "Mid Price Models"
        else:
            return "High Price Models"
    
    # Map fiModelDesc to categories
    model_category_mapping = model_avg_price.apply(categorize_price).to_dict()
    
    # Create ModelID to category mapping using fiModelDesc
    training_df["Model_Category"] = training_df["fiModelDesc"].map(model_category_mapping)
    modelid_to_category = training_df.set_index("ModelID")["Model_Category"].to_dict()
    
    return modelid_to_category, model_category_mapping

def categorize_model_id(df, modelid_to_category, model_category_mapping):
    """
    Categorizes ModelID based on precomputed price categories.
    If ModelID is not found, falls back to fiModelDesc categorization.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing ModelID and fiModelDesc columns.
    modelid_to_category (dict): Mapping of ModelID to Model_Category.
    model_category_mapping (dict): Mapping of fiModelDesc to Model_Category for fallback.
    
    Returns:
    pd.Series: A Series with categorized model price labels.
    """
    df["Predicted_Model_Category"] = df["ModelID"].map(modelid_to_category)
    
    # Handle missing ModelID by checking fiModelDesc mapping
    missing_mask = df["Predicted_Model_Category"].isna()
    df.loc[missing_mask, "Predicted_Model_Category"] = df.loc[missing_mask, "fiModelDesc"].map(model_category_mapping)
    
    # Handle any remaining missing values by assigning 'Unknown'
    df["Predicted_Model_Category"].fillna("Unknown", inplace=True)
    
    return df["Predicted_Model_Category"]



def extract_horsepower(df):
    """
    Extracts and imputes missing Horsepower values from fiProductClassDesc.
    Optimized for performance using vectorized operations.
    """
    def extract_numeric_range(value):
        """Extracts the average horsepower value from a range like '100 to 120 Horsepower'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:
            return float(numbers[0])
        return np.nan
    
    # Identify rows with Horsepower in fiProductClassDesc
    mask_horsepower = df['fiProductClassDesc'].str.contains("Horsepower", na=False)
    
    # Extract horsepower values
    df.loc[mask_horsepower, 'Horsepower_Unit_Type'] = 'Horsepower'
    df.loc[mask_horsepower, 'Extracted_Horsepower'] = df.loc[mask_horsepower, 'fiProductClassDesc'].apply(extract_numeric_range)
    
    # Handle 'Variable' and 'No' values in Engine_Horsepower
    df.loc[df['Engine_Horsepower'].isin(['Variable', 'No']), 'Engine_Horsepower'] = np.nan
    
    # Impute missing Engine_Horsepower values where Extracted_Horsepower is available
    df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])
    
    # Fill remaining NaN values with median horsepower
    #df['Engine_Horsepower_Imputed'].fillna(df['Engine_Horsepower_Imputed'].median(), inplace=True)
    df.drop(columns='Engine_Horsepower',inplace=True)
    
    return df

def preprocess_product_size(df):
    """
    Function to preprocess and impute missing ProductSize values based on fiProductClassDesc.
    
    Steps:
    1. Extract Product Type and Metric Tons / Horsepower from fiProductClassDesc.
    2. Identify Unit Type (Metric Tons, Horsepower, or Lb Operating Capacity).
    3. Convert Metric Tons / Horsepower to numerical values.
    4. Use Metric Tons to impute missing ProductSize values.
    """
    
    # Step 1: Extract Product Type and Size Descriptor
    def split_product_desc(desc):
        if pd.isna(desc):
            return pd.NA, pd.NA
        match = re.search(r'(.+?)-\s*([\d\.]+.*)', desc)
        if match:
            return match.group(1).strip(), match.group(2).strip()
        return desc.strip(), pd.NA  # If no match, return full description as product
    
    df[['Product_Type', 'Metric_Tons_HP']] = df['fiProductClassDesc'].apply(lambda x: pd.Series(split_product_desc(x)))
    
    # Step 2: Identify Unit Type
    def extract_unit_type(desc):
        if pd.isna(desc):
            return pd.NA
        if "Metric Tons" in desc:
            return "Metric Tons"
        elif "Horsepower" in desc:
            return "Horsepower"
        elif "Lb Operating Capacity" in desc:
            return "Lb Operating Capacity"
        return pd.NA
    
    df["Unit_Type"] = df["Metric_Tons_HP"].apply(extract_unit_type)
    
    # Step 3: Convert Metric Tons to numerical values
    def extract_numeric_range(value):
        """Extracts the average value from a range like '12.0 to 14.0 Metric Tons'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:  # If range exists, take the average
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:  # If only one number exists, use it
            return float(numbers[0])
        return np.nan
    
    df['Metric_Tons_Value'] = df['Metric_Tons_HP'].apply(lambda x: extract_numeric_range(x) if isinstance(x, str) else np.nan)
    
    # Step 4: Impute missing ProductSize values using Metric Tons
    def impute_product_size(row):
        if pd.isna(row['ProductSize']) and not pd.isna(row['Metric_Tons_Value']) and (row['Metric_Tons_Value']=='Metric Tons'):
            if row['Metric_Tons_Value']  <= 5:
                return 'Mini'
            elif 5 < row['Metric_Tons_Value'] <= 75:
                return 'Compact'
            elif 20 < row['Metric_Tons_Value'] <= 50:
                return 'Large / Medium'
            elif 75 < row['Metric_Tons_Value'] <= 200:
                return 'Medium'
            elif row['Metric_Tons_Value'] > 200:
                return 'Large'
        return row['ProductSize']
    
    df['ProductSize_Imputed'] = df.apply(impute_product_size, axis=1)
    df.drop(columns=['ProductSize','Metric_Tons_Value','Metric_Tons_HP','Unit_Type'],inplace=True)

    
    
    return df
def Enclosure_fun(df):
    list1=['EROPS','OROPS','EROPS AC']
    list2=['NO ROPS','None or Unspecified']
    df['Enclosure_cat']=0
    df.loc[df['Enclosure'].isin(list1), 'Enclosure_cat'] = 1
    df.loc[df['Enclosure'].isin(list2), 'Enclosure_cat'] = 0
    df.loc[df['Enclosure']=='EROPS w AC','Enclosure_cat']=2
    df.drop(columns='Enclosure',inplace=True)
    return df
# Define categories based on observed price trends
def categorize_hydraulics(value):
    if value in ["Missing", "Auxiliary", "Standard"]:
        return "Basic"
    elif value in ["2 Valve", "None or Unspecified", "3 Valve"]:
        return "Mid-Level"
    else:
        return "Advanced"



# Reload the dataset
ProductSize_path=r'C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/ProductSize_Null_fix_model.pkl'
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)[['SalesID', 'SalePrice',  'ModelID',
        'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 
       'Drive_System', 'Enclosure', 
        'Transmission', 'Turbocharged',  'Engine_Horsepower', 'Hydraulics',
         'Tire_Size',
       'Track_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls','fiModelDesc']]

# Convert 'saledate' to datetime and extract year, month, and day
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month
df['sale_day'] = df['saledate'].dt.day

# Apply categorization
df["Hydraulics_Category"] = df["Hydraulics"].apply(categorize_hydraulics)
# Apply categorization
df["Steering_Controls_Category"] = df["Steering_Controls"].apply(categorize_steering_controls)
# Apply categorization
df["Transmission_Category"] = df["Transmission"].apply(categorize_transmission)

# Apply categorization
df["Steering_Controls_Category"] = df["Steering_Controls"].apply(categorize_steering_controls_final)

df=Enclosure_fun(df)
df=extract_horsepower(df)

# Create mappings from training data only in saleprice 
modelid_to_category, model_category_mapping = create_model_category_mapping(df) # need to upload the modelid_to_category, model_category_mapping in the test

# Apply categorization to test data
df["Predicted_Model_Category"] = categorize_model_id(df, modelid_to_category, model_category_mapping) # need to upload the modelid_to_category, model_category_mapping in the test

# Compute the average price for each ProductGroup
# Apply categorization
df["Track_Type_Category"] = df["Track_Type"].apply(categorize_track_type)

product_group_avg_price = df.groupby("ProductGroup")["SalePrice"].mean()

# Map each ProductGroup to a price category
df["ProductGroup_Category"] = df["ProductGroup"].map(product_group_avg_price).apply(categorize_product_group) # need to upload the categorize_product_group in the test
df=preprocess_product_size(df)

# Define function to update 'YearMade' and create 'YearMade_Bucket'
def update_YearMade(df):
    print("update_YearMade START")
    
    # Compute median YearMade for each ModelID
    model_medians = df.loc[df['YearMade'] > 1000].groupby('ModelID')['YearMade'].median()
    
    # Update YearMade where it is 1000 using ModelID median
    df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
    
    # Compute overall median YearMade excluding 1000 values
    overall_median = df.loc[df['YearMade'] > 1000, 'YearMade'].median()
    
    # Replace any remaining 1000 values with overall median
    df['YearMade'].fillna(overall_median, inplace=True)
        
    # Bucketize YearMade
    bins = [0, 1980, 1995, 2005, 2010, 2025]
    labels = ["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]
    df['YearMade_Bucket'] = pd.cut(df['YearMade'], bins=bins, labels=labels, right=True)
    
    print("update_YearMade END")
    return df

# Apply YearMade update
df = update_YearMade(df)

#df.drop(columns=['fiProductClassDesc','Hydraulics','fiSecondaryDesc','auctioneerID','fiModelDesc','fiBaseModel','ProductGroupDesc','MachineID','ProductGroup'],inplace=True)


# Filter dataset to only include the last 5 years
recent_years = df['sale_year'].dropna().unique()
recent_years.sort()
#selected_years = recent_years[-5:]
#df_filtered = df[df['sale_year'].isin(selected_years)]
df_filtered=df.copy()

# Drop irrelevant columns: 'SalesID', 'saledate'
high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')

# Identify categorical columns
categorical_cols = df_filtered.select_dtypes(include=['object']).columns

df_filtered["YearMade_Bucket"] = df_filtered["YearMade_Bucket"].astype(str).fillna("Unknown")

# Define category order and apply Ordinal Encoding
year_bucket_encoder = OrdinalEncoder(
    categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025", "Unknown"]],
    handle_unknown="use_encoded_value",
    unknown_value=-1
)

df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])
# Apply Ordinal Encoding to 'YearMade_Bucket'
#year_bucket_encoder = OrdinalEncoder(categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]])
#df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])

# Identify high, moderate, and low cardinality categorical columns
high_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() >= 50]
moderate_cardinality_cols = [col for col in categorical_cols if 10 <= df_filtered[col].nunique() < 50]
low_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() < 10]

# Fill missing categorical values with "Unknown"
for col in categorical_cols:
    df_filtered[col] = df_filtered[col].fillna("Unknown")

# Encoding strategy

# Apply One-Hot Encoding (OHE) to low-cardinality categories
df_encoded = pd.get_dummies(df_filtered, columns=low_cardinality_cols, drop_first=True)

# Apply Ordinal Encoding to moderate-cardinality categories
if moderate_cardinality_cols:
    ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    df_encoded[moderate_cardinality_cols] = ord_encoder.fit_transform(df_encoded[moderate_cardinality_cols])

# Apply Frequency Encoding to high-cardinality categories
for col in high_cardinality_cols:
    freq_encoding = df_encoded[col].value_counts().to_dict()
    df_encoded[col] = df_encoded[col].map(freq_encoding)

# Drop rows with missing SalePrice (target variable)
df_encoded = df_encoded.dropna(subset=['SalePrice'])

# Split into features and target
X = df_encoded.drop(columns=['SalePrice'])
y = df_encoded['SalePrice']

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Display model performance
mae, rmse